# Prep variables for HydroTas 2020-2021 workplan:
- Skill assessment
  - rainfall, surface temp and surface wind over Australia region and Tasmania region
  - Assess skill as function of start month and ensemble size
- UNSEEN
  - Tasmanian rainfall and Melbourne surface temperature

In [ ]:
import glob
import cftime
import numpy as np
import xarray as xr
import multiyear as my
from dask.distributed import Client
from dask_jobqueue import PBSCluster

In [ ]:
walltime = '02:00:00'
cores = 12
memory = '24GB'

cluster = PBSCluster(processes=1,
                     walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),
                                '-l mem='+str(memory),
                                '-P ux06',
                                '-l jobfs=100GB',
                                '-l storage=gdata/xv83+gdata/v14+scratch/v14'],
                     local_directory='$PBS_JOBFS',
                     header_skip=["select"])

In [ ]:
cluster.scale(jobs=4)
client = Client(cluster)
client

# Global parameters

In [ ]:
REGIONS = {'AUS': [-44, -10, 112, 155],
           'TAS': [],
           'MEL': []}